In [2]:
import numpy as np

In [ ]:
def lu_decomposition(A, dU=None):
    n = A.shape[0]
    
    if dU is None:
        dU = np.ones(n)
    
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    for i in range(n):
        U[i, i] = dU[i]
    
    L[0, 0] = A[0, 0] / U[0, 0]
    
    for j in range(1, n):
        U[0, j] = A[0, j] / L[0, 0]
    
    for i in range(1, n):
        L[i, 0] = A[i, 0] / U[0, 0]
    
    for p in range(1, n):
        sum_l = 0
        for k in range(p):
            sum_l += L[p, k] * U[k, p]
        L[p, p] = (A[p, p] - sum_l) / U[p, p]  
        # L[1,1] = (7-1*1)/3 = 2
        # L[2,2] = (12.5-3*0)/4 = 3.125
        
        for j in range(p+1, n):
            sum_u = 0
            for k in range(p):
                sum_u += L[p, k] * U[k, j]
            U[p, j] = (A[p, j] - sum_u) / L[p, p]
    
        for i in range(p+1, n):
            sum_l = 0
            for k in range(p):
                sum_l += L[i, k] * U[k, p]
            L[i, p] = (A[i, p] - sum_l) / U[p, p]
    
    return L, U


def find_determinant(A, dU):
    # After the decomposition, A = LU and det(A) = det(L) * det(U) 
    # Therefore det(A) = det(L) * det(U) = product of the diagonal elements of U and L
    return np.prod([A[i][i] for i in range(A.shape[0])]) * np.prod([dU[i] for i in range(len(dU))])



In [ ]:
import numpy as np

n = 3

A = np.array([
    [4, 2, 3],
    [2, 7, 5.5],
    [6, 3, 12.5]
])

# L = np.array([
#     [2, 0, 0],
#     [1, 2, 0],
#     [3, 0, 2]
# ])

# U = np.array([
#     [2, 1, 1.5],
#     [0, 3, 2],
#     [0, 0, 4]
# ])

Ainit = A.copy()

valid = True
dU = [2, 3, 4]

for k in range(1, n+1):
    Ak = A[:k, :k]
    det = np.linalg.det(Ak)
    if abs(det) < 1e-8:
        valid = False
        print("Nu se poate calcula o descompunere LU")
        break

if valid:
    L, U = lu_decomposition(A, dU)
    print("L =\n", L)
    print("U =\n", U)
    print(find_determinant(L, dU))
    


NameError: name 'L' is not defined